In [1]:
import os
import torch
import torchvision

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from model import UNetModel
from pathlib import Path
from argparse import Namespace

In [2]:
um = UNetModel({}, '')

2020-09-28 00:26:54,254 [MainThread] INFO ConfigLoader - Using 'cpu' device


In [3]:
um.model

ResidualUNet3D(
  (encoders): ModuleList(
    (0): Encoder(
      (basic_module): ExtResNetBlock(
        (conv1): SingleConv(
          (groupnorm): GroupNorm(1, 1, eps=1e-05, affine=True)
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (ReLU): ReLU(inplace=True)
        )
        (conv2): SingleConv(
          (groupnorm): GroupNorm(8, 16, eps=1e-05, affine=True)
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
          (ReLU): ReLU(inplace=True)
        )
        (conv3): SingleConv(
          (groupnorm): GroupNorm(8, 16, eps=1e-05, affine=True)
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        )
        (non_linearity): ReLU(inplace=True)
      )
    )
    (1): Encoder(
      (pooling): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (basic_module): ExtResNetBlock(


In [4]:
from torchsummary import summary

In [6]:
summary(UNetModel({}, '').model, (1, 128, 128, 128))

2020-09-28 00:28:09,428 [MainThread] INFO ConfigLoader - Using 'cpu' device


IndexError: tuple index out of range

In [2]:
algo = "conv"
architecture = "arch_1"

data_path = '/scratch/ns4486/numerical-relativity-interpolation/Proca_fiducial_scaled_cropped.hdf5'
# data_path = "/scratch/prs392/research_internship/data/sketch_recognition/tu_berlin/png_split/"
checkpoint_path = '/scratch/ns4486/checkpoints'

experiment_name = 'tuning'

d = os.path.join(checkpoint_path, experiment_name)
Path(d).mkdir(parents=True, exist_ok=True)
versions = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]

versions = sorted(versions)

In [3]:
checkpoint_path

'/scratch/ns4486/checkpoints'

In [4]:
from tensorboard.backend.event_processing import event_accumulator
import numpy as np

def _load_run(path):
    event_acc = event_accumulator.EventAccumulator(path)
    event_acc.Reload()
    data = {}

    for tag in sorted(event_acc.Tags()["scalars"]):
        x, y = [], []

        for scalar_event in event_acc.Scalars(tag):
            x.append(scalar_event.step)
            y.append(scalar_event.value)

        data[tag] = (np.asarray(x), np.asarray(y))
    return data

In [5]:
from pytorch_lightning.core.saving import load_hparams_from_yaml

list_existing_hparams = []
list_of_val_loss = []

for version in versions:
    hparam_path = os.path.join(checkpoint_path, experiment_name, version, 'hparams.yaml')
    hparams_new = load_hparams_from_yaml(hparam_path)

    print(hparams_new)
    if 'val_loss' in _load_run(os.path.join(checkpoint_path, experiment_name, version)).keys():
        print(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

        list_existing_hparams.append(hparams_new)
        list_of_val_loss.append(min(_load_run(os.path.join(checkpoint_path, experiment_name, version))['val_loss'][1]))

{'batch_size': 16}
{'batch_size': 16}
0.02971024066209793


/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/core/saving.py:335: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  tags = yaml.load(fp)


In [6]:
seed_everything(123)

parameterization = {
    'batch_size' : 16
}

print(parameterization)

hparams = Namespace(**parameterization)
model = PlaceholderModel(hparams, data_path)
logger = TensorBoardLogger(checkpoint_path, name=experiment_name)
checkpoint_callback = ModelCheckpoint(
    filepath=None,
    save_top_k=5,
    save_last = True,
    verbose=False,
    monitor='val_loss',
    mode='min',
    prefix=''
)
trainer = Trainer(
     logger = logger, 
     default_root_dir=checkpoint_path,
     checkpoint_callback = checkpoint_callback,
     max_epochs = 5,
     gpus = -1,
#      distributed_backend = 'dp',
     val_check_interval=0.10,
     log_save_interval=1,
     row_log_interval=1,
     fast_dev_run=False
)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


{'batch_size': 16}


/ext3/miniconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | Sequential | 3 M   
1 | decoder   | Sequential | 4 M   
2 | criterion | MSELoss    | 0     


Traceback (most recent call last):
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/ext3/miniconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


Saving latest checkpoint..


1